[View in Colaboratory](https://colab.research.google.com/github/hyerim1048/100_pytorch/blob/master/_1_movies_rating.ipynb)

In [0]:
!pip install pandas torch

In [28]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.1MB/s 
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [0]:
!git clone https://github.com/e9t/nsmc.git

In [0]:
!apt-get update

!apt-get install g++ openjdk-8-jdk python-dev python3-dev

!pip3 install JPype1-py3

!pip3 install konlpy

!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

In [0]:
def read_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]   # header 제외
    return data
train_data = read_data('./nsmc/ratings_train.txt')
test_data = read_data('./nsmc/ratings_test.txt')

In [0]:
print(len(train_data))      
print(len(train_data[0]))   
print(len(test_data))       
print(len(test_data[0])) 

150000
3
50000
3


In [0]:
from collections import defaultdict
import math
import time
import random
import numpy as np
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from collections import deque

In [8]:
from konlpy.tag import Twitter
pos_tagger = Twitter() #형태소 분석기
def tokenize(doc):
    # 형태소 분석후 [영화, Noun]을 "영화/Noun" 형태로 바꿔주는 function
    # norm, stem은 optional
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

#모든 train data와 test data 에 적용
train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
test_docs = [(tokenize(row[1]), row[2]) for row in test_data]



2194536
48765


In [20]:
tokens = [t for d in train_docs+test_docs for t in d[0]]
print(len((tokens))) # token 총 개수
print(len(set(tokens))) # token unique 개수

2929087
55824


In [0]:
# test and train set
sentences = [train_docs[i][0] for i in range(len(train_docs))]
for s in test_docs:
  sentences.append(s[0])

In [0]:
# word to index
w2i = defaultdict(lambda: len(w2i))

def word_to_idx(tag_list):
    for tags in tag_list:
        yield [w2i[tag] for tag in tags]
data = word_to_idx(sentences)

train= list(data)
i2w = {v: k for k, v in w2i.items()}

In [0]:
# hyper parameter - have to check
emb_size = len(set(tokens))
emb_dimension = 200
batch_size = 128
window_size = 5
iteration=1
initial_lr=0.025
min_count=3
sentence_count= len(sentences)
sentence_length= len((tokens)) # 3  이하인 거는 안지움

In [0]:
def evaluate_pair_count(window_size):
    return sentence_length * (2 * window_size - 1) - (sentence_count - 1) * (1 + window_size) * window_size


In [36]:
pair_count = evaluate_pair_count(window_size) # total pair
batch_count = iteration * pair_count / batch_size # for 1 iteration how many batch iteration is needed
process_bar = tqdm(range(int(batch_count)))
word_pair_catch = deque()



  0%|          | 0/159076 [00:00<?, ?it/s]

In [26]:
for i in process_bar:

20361813

In [0]:
# batch pair 만드는 함수 고치기
#  sentence  순서대로?
def get_batch_pairs(batch_size, window_size):
    while len(word_pair_catch) < batch_size:

        word_ids = []
        for word in sentence.strip().split(' '):
            try:
                word_ids.append(self.word2id[word])
            except:
                continue
        for i, u in enumerate(word_ids):
            for j, v in enumerate(
                    word_ids[max(i - window_size, 0):i + window_size]):
                assert u < self.word_count
                assert v < self.word_count
                if i == j:
                    continue
                self.word_pair_catch.append((u, v))
    batch_pairs = []
    for _ in range(batch_size):
        batch_pairs.append(self.word_pair_catch.popleft())
    return batch_pairs